In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/squad-20/train-v2.0.json
/kaggle/input/squad-20/dev-v2.0.json


In [2]:
import json

def extract_squad_data(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as file:
        squad_data = json.load(file)
    
    data = squad_data['data']
    extracted_data = []

    for entry in data:
        paragraphs = entry['paragraphs']
        for paragraph in paragraphs:
            context = paragraph['context']
            qas = paragraph['qas']
            for qa in qas:
                question = qa['question']
                is_impossible = qa['is_impossible']
                
                if not is_impossible:
                    answers = [answer['text'] for answer in qa['answers']]
                else:
                    answers = []

                extracted_data.append({
                    'context': context,
                    'question': question,
                    'answers': answers,
                    'is_impossible': is_impossible
                })

    return extracted_data

# Example usage:
json_file_path = '/kaggle/input/squad-20/train-v2.0.json'
train_data = extract_squad_data(json_file_path)
val_data = extract_squad_data('/kaggle/input/squad-20/dev-v2.0.json')
# You can now use the processed_data for further preprocessing or analysis.


In [3]:
val_data = [item for item in val_data if not item['is_impossible']]
train_data = [item for item in train_data if not item['is_impossible']]

In [4]:
val_data[231]

{'context': 'If a problem X is in C and hard for C, then X is said to be complete for C. This means that X is the hardest problem in C. (Since many problems could be equally hard, one might say that X is one of the hardest problems in C.) Thus the class of NP-complete problems contains the most difficult problems in NP, in the sense that they are the ones most likely not to be in P. Because the problem P = NP is not solved, being able to reduce a known NP-complete problem, Π2, to another problem, Π1, would indicate that there is no known polynomial-time solution for Π1. This is because a polynomial-time solution to Π1 would yield a polynomial-time solution to Π2. Similarly, because all NP problems can be reduced to the set, finding an NP-complete problem that can be solved in polynomial time would mean that P = NP.',
 'question': 'If P = NP is unsolved, and reduction is applied to a known NP-complete problem vis a vis Π2 to  Π1, what conclusion can be drawn for Π1?',
 'answers': ['ther

In [5]:
train_content = []
train_q = []
train_a = []

for block in train_data:
    train_content.append(block['context'])
    train_q.append(block['question'])
    train_a.append(block['answers'])
    
    
val_content = []
val_q = []
val_a = []

for block in val_data:
    val_content.append(block['context'])
    val_q.append(block['question'])
    val_a.append(block['answers'])

In [6]:
# onet_content = train_content[:1000]
# onet_q = train_q[:1000]
# onet_a = train_a[:1000]
# onev_content = val_content[:100]
# onev_q = val_q[:100]
# onev_a = val_a[:100]

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function for text preprocessing
def preprocess_text(data):
    # Convert to lowercase
    data2 = []
    for text in data:
        text = text.lower()

        # Remove punctuation and digits
#         text = text.translate(str.maketrans('', '', string.punctuation + string.digits))

        # Tokenization
        tokens = word_tokenize(text)

        # Remove stop words
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Lemmatization
    #     lemmatizer = WordNetLemmatizer()
    #     tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Join the tokens back into a string
        preprocessed_text = ' '.join(tokens)
        data2.append(preprocessed_text)

    return data2



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# onet_q[0][0]

In [9]:
len(train_content)

86821

In [10]:
train_a = [item for sublist in train_a for item in sublist]
val_a = [item for sublist in val_a for item in sublist]

In [11]:
onet_content = preprocess_text(train_content)
onet_q = preprocess_text(train_q)
onet_a = preprocess_text(train_a)

onev_content = preprocess_text(val_content)
onev_q = preprocess_text(val_q)
onev_a = preprocess_text(val_a)


In [12]:
len(onet_a)

86821

In [13]:

# from transformers import GPT2Tokenizer

# # Load GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# # Define a function to tokenize, convert text to indices, and truncate to the first 1021 tokens
# def tokenize_and_convert_to_indices(data_list, max_length):
#     tokenized_data_list = []
#     for article in data_list:
#         # Tokenize and convert to indices
#         article_tokens = tokenizer.encode(article, add_special_tokens=True)
# #         highlights_tokens = tokenizer.encode(highlights, add_special_tokens=True)

#         # Truncate to the first 1021 tokens
#         article_tokens = article_tokens[:max_length]
# #         highlights_tokens = highlights_tokens[:max_length+3]

#         # Append to the tokenized_data_list
#         tokenized_data_list.append(article_tokens)

#     return tokenized_data_list

# # Apply tokenization and truncation to your datasets
# max_length = 1021
# tokenized_train_c = tokenize_and_convert_to_indices(onet_content, 952)
# tokenized_train_q = tokenize_and_convert_to_indices(onet_q, 65)
# tokenized_train_a = tokenize_and_convert_to_indices(onet_a, 1024)

In [14]:
# tokenized_val_c = tokenize_and_convert_to_indices(onev_content, 952)
# tokenized_val_q = tokenize_and_convert_to_indices(onev_q, 65)
# tokenized_val_a = tokenize_and_convert_to_indices(onev_a, 1024)

In [15]:
from transformers import GPT2Tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# context give answers for the following question
# Tokenize the word "summarize"
tokens = tokenizer.encode("context give answer for the following question")
print(tokens)

[22866, 1577, 3280, 329, 262, 1708, 1808]


In [16]:
# context  : 952
# prompt   : 7
# question : 65

In [17]:
from transformers import GPT2Tokenizer
import torch
from torch.nn.utils.rnn import pad_sequence

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Define a pad token and add it to the tokenizer
pad_token = tokenizer.eos_token
tokenizer.add_tokens([pad_token])

# Define a function to tokenize, convert text to indices, and pad sequences
def tokenize_and_pad(data_list, max_article_length=1021):
    tokenized_data_list = []
    for article in data_list:
        # Tokenize and convert to indices
        article_tokens = tokenizer.encode(article, add_special_tokens=True)
#         highlights_tokens = tokenizer.encode(highlights, add_special_tokens=True)

        # Pad sequences to specified lengths
        padded_article_tokens = torch.tensor(article_tokens + [tokenizer.convert_tokens_to_ids(pad_token)] * (max_article_length - len(article_tokens)))
#         padded_highlights_tokens = torch.tensor(highlights_tokens + [tokenizer.convert_tokens_to_ids(pad_token)] * (max_highlights_length - len(highlights_tokens)))
        truncated_article_tokens = padded_article_tokens[:max_article_length]

        # Append to the tokenized_data_list only if both token lists are not empty
        if len(article_tokens) > 0:
            tokenized_data_list.append(padded_article_tokens)
        
    return tokenized_data_list

# Apply tokenization and padding to your datasets
# max_article_length = 1021
# max_highlights_length = 1024
# tokenized_train_c = tokenize_and_pad(tokenized_train_c, max_article_length)
# tokenized_test_data_list = tokenize_and_pad(ptest_data_list, max_article_length)
# tokenized_val_data_list = tokenize_and_pad(pval_data_list, max_article_length)
tokenized_train_c = tokenize_and_pad(onet_content, 952)
tokenized_train_q = tokenize_and_pad(onet_q, 65)
tokenized_train_a = tokenize_and_pad(onet_a, 1024)
tokenized_val_c = tokenize_and_pad(onev_content, 952)
tokenized_val_q = tokenize_and_pad(onev_q, 65)
tokenized_val_a = tokenize_and_pad(onev_a, 1024)


In [18]:
tokenized_trainqqq = []

for padded_article_tokens in tokenized_train_q:
    # Truncate article tokens if greater than max_article_length
    truncated_article_tokens = padded_article_tokens[:65]
    tokenized_trainqqq.append(truncated_article_tokens)


In [19]:
# Convert the lists to PyTorch tensors
c_train = torch.stack(tokenized_train_c)
q_train = torch.stack(tokenized_trainqqq)
a_train = torch.stack(tokenized_train_a)
c_val = torch.stack(tokenized_val_c)
q_val = torch.stack(tokenized_val_q)
q_val = torch.stack(tokenized_val_a)

In [20]:
len(q_train[19339])

65

In [21]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, AdamW

# Load GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
gpt2_model = GPT2LMHeadModel.from_pretrained(model_name)

# Define the number of prompts and embedding size
num_prompts = 7  # "summarize the following text"
embedding_size = 768

# Define a specific sentence
sentence = "context give answer for the following question"

# Tokenize the sentence
input_ids = tokenizer.encode(sentence, return_tensors='pt')

# Get the embeddings for the input_ids from the GPT-2 model
gpt2_embeddings = gpt2_model.transformer.wte(input_ids)

# Create an embedding layer for soft prompts and initialize with the sentence embeddings
soft_prompt_embeddings = nn.Embedding(num_prompts, embedding_size)
soft_prompt_embeddings.weight.data.copy_(gpt2_embeddings.squeeze(0))

# Concatenate soft prompt embeddings at the beginning of the input sequence
class GPT2WithPromptTuning(nn.Module):
    def __init__(self, gpt2_model, soft_prompt_embeddings):
        super(GPT2WithPromptTuning, self).__init__()
        self.gpt2_model = gpt2_model
        self.soft_prompt_embeddings = soft_prompt_embeddings
    
    def forward(self, input_c, input_q, soft_prompt_c, soft_prompt_q):
        # Get the embeddings for the input_ids from the GPT-2 model
#         gpt2_outputs = self.gpt2_model(input_ids)
#         gpt2_embeddings = gpt2_outputs['last_hidden_state']
        c_embeddings = self.gpt2_model.transformer.wte(input_c)
        q_embeddings = self.gpt2_model.transformer.wte(input_q)

        # Get the embeddings for the soft prompts
        soft_prompt_c = self.soft_prompt_embeddings(soft_prompt_c)
        soft_prompt_q = self.soft_prompt_embeddings(soft_prompt_q)

        # Concatenate the embeddings
        embeddings = torch.cat([soft_prompt_c, c_embeddings ,soft_prompt_q, q_embeddings], dim=0)
        
        # Pass the concatenated embeddings through the GPT-2 model
        outputs = self.gpt2_model(inputs_embeds=embeddings)
        
        return outputs 

In [22]:
# Initialize the model
model = GPT2WithPromptTuning(gpt2_model, soft_prompt_embeddings)

# Freeze GPT-2 model weights
for param in model.gpt2_model.parameters():
    param.requires_grad = False

# Define hyperparameters
batch_size = 8
epochs = 5
learning_rate = 2e-5
gradient_clip_value = 1.0

# Define optimizer and criterion
optimizer = torch.optim.AdamW(model.soft_prompt_embeddings.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=-100)

# Example data (replace with your own dataset)
# input_ids = torch.randint(0, 100, (1023,))  # Example input sequence
soft_prompt_c = torch.tensor([0])
soft_prompt_q = torch.tensor([1, 2, 3, 4, 5, 6])
# target_ids = torch.randint(0, 100, (1024,))  # Example input sequence


In [23]:
from tqdm import tqdm
import nltk
from nltk.translate.bleu_score import sentence_bleu
# from nltk.translate.meteor_score import meteor_score
# from nltk.translate.rouge import Rouge

# Download necessary resources for nltk (run this once)
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download("universal_tagset")
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
c = c_train[:500]
q = q_train[:500]
a = a_train[:500]

In [25]:
from tqdm import tqdm
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU
model.to(device)

# Define device (assuming you have a CUDA-enabled GPU)

# Move optimizer and criterion to GPU
optimizer = torch.optim.AdamW(model.soft_prompt_embeddings.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss(ignore_index=-100).to(device)

# Initialize Weights and Biases
wandb.init(project='SoftPrompt on GPT-2 for SQUAD', name='2', config={'learning_rate': learning_rate})

# Training loop
for epoch in range(epochs):
    # Create a tqdm progress bar for the training data
    data_iterator = tqdm(zip(c_train, q_train, a_train), desc=f'Epoch {epoch + 1}', total=len(c_train))
    
    for c_train, q_train, a_train in data_iterator:
        optimizer.zero_grad()

        # Move input and target tensors to GPU
        c_train, q_train, a_train = c_train.to(device), q_train.to(device), a_train.to(device)
        
        # Assuming you have a soft_prompt_ids for each training instance
        # If not, you might need to modify this part accordingly
        outputs = model(c_train, q_train, soft_prompt_c.to(device), soft_prompt_q.to(device))
        logits = outputs.logits if hasattr(outputs, "logits") else outputs.last_hidden_state

        loss = criterion(logits, a_train)
        loss.backward()

        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_value)

        optimizer.step()

        # Update the progress bar description with the current loss
        data_iterator.set_postfix(loss=loss.item())

#     # Validation loop
    model.eval()
    val_losses = []
    with torch.no_grad():
        for input_ids_val, target_ids_val in zip(input_ids_val, target_ids_val):
            input_ids_val, target_ids_val = input_ids_val.to(device), target_ids_val.to(device)
            outputs_val = model(input_ids_val, soft_prompt_ids.to(device))
            logits_val = outputs_val.logits if hasattr(outputs_val, "logits") else outputs_val.last_hidden_state
            loss_val = criterion(logits_val, target_ids_val)
            val_losses.append(loss_val.item())
            # Convert tensor predictions and references to lists
            predictions = logits_val.argmax(dim=-1).squeeze(0).tolist()
            references = target_ids_val.squeeze(0).tolist()

            # BLEU Score
            bleu_score = sentence_bleu([references], predictions)
            print(f"BLEU Score: {bleu_score}")
    avg_val_loss = sum(val_losses) / len(val_losses)
    wandb.log({"epoch": epoch + 1, "train_loss": loss.item()})

    # Set the model back to training mode
#     model.train()

# Close the tqdm progress bar
data_iterator.close()


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 1:  18%|█▊        | 15317/86821 [1:03:52<4:58:09,  4.00it/s, loss=0.0863]


KeyboardInterrupt: 